## Launch Simulator

In [1]:
# launch simulation window
import numpy as np
from bullet_utils.env import BulletEnvWithGround
from robot_properties_solo.solo8wrapper import Solo8Config, Solo8Robot

# Create the simulation environment with a ground.
env = BulletEnvWithGround()

# Create a robot instance and add it to the simulation.
robot = Solo8Robot()
env.add_robot(robot)

# A class to simulate the DG Head with the simulator.
class SoloSimHead:
    def __init__(self, env, robot):
        self._env = env
        self._robot = robot
        
        # Define the sensor values.
        self._sensor_joint_positions = np.zeros(8)
        self._sensor_joint_velocities = np.zeros(8)
        self._sensor_slider_positions = np.zeros(4)
        
        self._sensor_slider_positions = np.zeros(4)
        
        self._sensor_imu_gyroscope = np.zeros(3)
        
        # Utility for vicon class.
        self._sensor__vicon_base_position = np.zeros(7)
        self._sensor__vicon_base_velocity = np.zeros(6)
        
        # Controls.
        self._control_ctrl_joint_torques = np.zeros(8)
        
    def read(self):
        q, dq = self._robot.get_state()
        self._sensor_joint_positions[:] = q[7:]
        self._sensor_joint_velocities[:] = dq[6:]
        
        for i, l in enumerate(['a', 'b', 'c', 'd']):
            self._sensor_slider_positions[i] = robot.get_slider_position(l)
        
        self._sensor_imu_gyroscope[:] = dq[3:6]
        
        self._sensor__vicon_base_position[:] = q[:7]
        self._sensor__vicon_base_velocity[:] = dq[:6]
        
    def write(self):
        self._robot.send_joint_command(self._control_ctrl_joint_torques)
        self._env.step(sleep=False)
        
    def get_sensor(self, sensor_name):
        return self.__dict__['_sensor_' + sensor_name]
    
    def set_control(self, control_name, value):
        self.__dict__['_control_' + control_name][:] = value
        
    def reset_state(self, q, dq):
        self._robot.reset_state(q, dq)

# Instantiate the head class and use it similar to the DG head you get
# on the real robot.
head = SoloSimHead(env, robot)

## Constants

In [2]:
# joints point inwards, towards each other
# not really lying down, but will be after 500 ticks
lying_down_start = np.array([0.0, 0, 0.4,  # x,y,z
                         0, 0, 0, 1,  # quaterion
                         0.8, -1.6, 0.8, -1.6, -0.8, 1.6, -0.8, 1.6]  # joint positions
)


# unused 
standing_start = np.array([
                    0.2, 0, 0.4, 
                    0, 0, 0, 1, 
                    0.186, -0.415, 0.185, -0.415, -0.192, 0.416, -0.192, 0.415]
)
# # reset_state(...) template
# head.reset_state(Solo8Config.q0, Solo8Config.v0)
# head.reset_state(custom_start, Solo8Config.v0)


# dampening constant
D = 0.1

# motor constant
K = 5

# take single angle and "map" to target joint positions
angle_adjust = np.array([1,-2,1,-2,-1,2,-1,2]) * np.pi

## Functions

In [3]:
def reset(start=None):
    head.reset_state(lying_down_start if start is None else start, Solo8Config.v0)

    for _ in range(500):  # lay robot down
        tau = np.zeros((8))
        head.set_control('ctrl_joint_torques', tau)
        head.write()

        
# single frame, have to loop over this function
def bounce_in_place_hardcoded_single_frame(up):
    joint_pos = head.get_sensor('joint_positions')
    joint_velocities = head.get_sensor('joint_velocities')
    
    a = np.pi*np.array([1.0/6, -1.0/3]*2)
    target_up = np.concatenate((a, -1*a))
    
    a = np.pi*np.array([0.5, -1]*2)
    target_down = np.concatenate((a, -1*a))
    
    if up:
        if np.sum(np.abs(joint_pos - target_up)) < 0.5:
            up = False
    
        tau = K*(target_up - joint_pos) - D*joint_velocities
            
    else:
        if np.sum(np.mod(np.abs(joint_pos - target_down), 2*np.pi)) < 0.5:
            up = True
        tau = -D/4*joint_velocities  # let down slowly 
        # this was because dampening was too high, simulation would start "flying"
                
    return tau, up


# multiple frames: goes up and down, call as many times as you want it to bounce
def bounce_in_place_hardcoded():
    a = np.pi*np.array([1.0/6, -1.0/3]*2)  # same as angle adjust, angle adjust was written later
    target_up = np.concatenate((a, -1*a))
    
    a = np.pi*np.array([0.5, -1]*2)
    target_down = np.concatenate((a, -1*a))
    
    while True:
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        
        if np.sum(np.mod(np.abs(joint_pos - target_up), 2*np.pi)) < 0.5:
            break
    
        tau = K*(target_up - joint_pos) - D*joint_velocities
        
        head.set_control('ctrl_joint_torques', tau)
        head.write()
            
    while True:
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        
        if np.sum(np.mod(np.abs(joint_pos - target_down), 2*np.pi)) < 0.5:
            break
            
        tau = -D/4*joint_velocities  # let down slowly
    
        head.set_control('ctrl_joint_torques', tau)
        head.write()
        

# goes up and stays there until slider a > 0.5
def stay_still():
    a = np.pi*np.array([1.0/6, -1.0/3]*2)
    target_up = np.concatenate((a, -1*a))
    
    a = np.pi*np.array([0.5, -1]*2)
    target_down = np.concatenate((a, -1*a))
    
    while True:  # stand up
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        
        if np.sum(np.mod(np.abs(joint_pos - target_up), 2*np.pi)) < 0.5:
            break
        
        tau = K*(target_up - joint_pos) - D*joint_velocities
        
        head.set_control('ctrl_joint_torques', tau)
        head.write()
    
    
    while True:  # stay still
        head.read()
        slider_a_pos = head.get_sensor('slider_positions')[0]
        
        if slider_a_pos > 0.5: # don't move until slider > 0.5
            break
            
    
    while True:  # get down
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        
        if np.sum(np.mod(np.abs(joint_pos - target_down), 2*np.pi)) < 0.5:
            break
            
        tau = -D/4*joint_velocities  # let down slowly
    
        head.set_control('ctrl_joint_torques', tau)
        head.write()

        
def move_with_slider():
    while 1:  
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        slider_pos = head.get_sensor('slider_positions')
        A = slider_pos[0]
                    

        target = angle_adjust*A/2  # div by 2 (/2) lets you use the whole slider; no longer restricted to lower half
            
        tau = K*(target - joint_pos) - D*joint_velocities
        
        head.set_control('ctrl_joint_torques', tau)
        head.write()


def move_leg_forward():
    a = np.pi*np.array([1/6, -1/3]*2)
    target_up = np.concatenate((a, -1*a))
    
    a = np.pi*np.array([0.5, -1]*2)

    target_down = np.concatenate((a, -1*a))
    while True:  # up
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        
        if np.sum(np.abs(joint_pos - target_up)) < 0.5:
            break
        
        tau = K*(target_up - joint_pos) - D*joint_velocities
        
        head.set_control('ctrl_joint_torques', tau)
        head.write()
    
    while True:  # down
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        slider_a_pos = head.get_sensor('slider_positions')[0]
        
        if slider_a_pos > 0.5: # manual exit
            break
        
        if np.sum(np.mod(np.abs(joint_pos - target_down), 2*np.pi)) < 0.5:
            break
        
        tau = -D/4*joint_velocities  # let down slowly
    
        head.set_control('ctrl_joint_torques', tau)
        head.write()


def bounce_in_place():
    target_up = angle_adjust * 1/6
    
    head.read()    
    joint_positions = head.get_sensor('joint_positions')
    
    L = np.linspace(joint_positions, target_up, num=2000)
    
    for target in L:
        head.read()
        
        joint_positions = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        slider_positions = head.get_sensor('slider_positions')
        
        tau = K*(target - joint_positions) - D*joint_velocities
        
        head.set_control('ctrl_joint_torques', tau)
        head.write()
            
    target_down = angle_adjust * 0.4
    
    head.read()
    joint_positions = head.get_sensor('joint_positions')
    print(joint_positions- target_up)
    L = np.linspace(joint_positions, target_down, num=2000)
    
    for target in L:
        head.read()
        
        joint_positions = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        slider_positions = head.get_sensor('slider_positions')
        
        tau = K/10*(target - joint_positions) - D*joint_velocities
        
        head.set_control('ctrl_joint_torques', tau)
        head.write()

        
def up_then_slider():  # up, then use slider 
    reset()

    head.read()
    joint_positions = head.get_sensor('joint_positions')
    target_up = angle_adjust * head.get_sensor('slider_positions')[0]/2
    L = np.linspace(joint_positions, target_up, num=2000)

    for target in L:
        head.read()
        joint_positions = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')

        tau = K*(target - joint_positions) - D*joint_velocities

        head.set_control('ctrl_joint_torques', tau)
        head.write()

    while 1:  
        head.read()
        joint_pos = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')
        slider_pos = head.get_sensor('slider_positions')

        A = slider_pos[0]/2
        B_t = slider_pos[1] * angle_adjust/2


        target = angle_adjust*A
    #     target[3:7] = B_t[3:7]
    #     target[7:] = B_t[7:]
    #     target[2:4] = B_t[2:4]
        target[2:6] = B_t[2:6]

    #     print(target, end='\r')

        tau = K*(target - joint_pos) - D*joint_velocities

        head.set_control('ctrl_joint_torques', tau)
        head.write()
        
        
def up_then_down():  # up, then down; linspace/slider-esque 
    reset()  # to laying down

    target_up = angle_adjust * 0.01

    head.read()
    joint_positions = head.get_sensor('joint_positions')

    L = np.linspace(joint_positions, target_up, num=2000)

    for target in L:
        head.read()
        joint_positions = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')

        tau = K*(target - joint_positions) - D*joint_velocities

        head.set_control('ctrl_joint_torques', tau)
        head.write()

    target_down = angle_adjust * 0.49

    head.read()
    joint_positions = head.get_sensor('joint_positions')

    L = np.linspace(joint_positions, target_down, num=2000)

    for target in L:
        head.read()
        joint_positions = head.get_sensor('joint_positions')
        joint_velocities = head.get_sensor('joint_velocities')

        tau = K*(target - joint_positions) - D*joint_velocities

        head.set_control('ctrl_joint_torques', tau)
        head.write()




## Run

In [ ]:
reset()

head.read()
joint_positions = head.get_sensor('joint_positions')
target_up = angle_adjust * head.get_sensor('slider_positions')[0]/2
L = np.linspace(joint_positions, target_up, num=2000)

for target in L:
    head.read()
    joint_positions = head.get_sensor('joint_positions')
    joint_velocities = head.get_sensor('joint_velocities')

    tau = K*(target - joint_positions) - D*joint_velocities

    head.set_control('ctrl_joint_torques', tau)
    head.write()

while 1:
    head.read()
    joint_pos = head.get_sensor('joint_positions')
    joint_velocities = head.get_sensor('joint_velocities')
    slider_pos = head.get_sensor('slider_positions')

#     A = slider_pos[0] * angle_adjust/2
#     B = slider_pos[1] * angle_adjust/2
#     C = slider_pos[2] * angle_adjust/2
#     D = slider_pos[3] * angle_adjust/2

#     target = angle_adjust * A
# #     target[2:4] = B[2:4]
    target[2:6] = B[2:6]

#     target = np.array([A[0:2], B[2:4], C[4:6], D[6:8]]).reshape((-1))

    tau = K*(target - joint_pos) - D*joint_velocities

    head.set_control('ctrl_joint_torques', tau)
    head.write()